In [1]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [2]:
!wget https://www.dropbox.com/s/4jw31k5mlzcmgis/genres.tar.gz?dl=0

--2023-07-25 10:18:28--  https://www.dropbox.com/s/4jw31k5mlzcmgis/genres.tar.gz?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/4jw31k5mlzcmgis/genres.tar.gz [following]
--2023-07-25 10:18:28--  https://www.dropbox.com/s/raw/4jw31k5mlzcmgis/genres.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc89d1cc966b106f9fc433469516.dl.dropboxusercontent.com/cd/0/inline/CAiBAX-iU6ncQ_H_Gp7meyAfm1gT33Msp7muonyRBtr6Xv2UzJa724skw1utaIKinFl1AV0enYafXJukW5XB9N_270wtVDMjWUkmaRj4rbx8_Gj7sEqTKRWZc3W-P94UivJttdMar7h92_cBzlQwO8XK839_eev6b72oKHsn6o112A/file# [following]
--2023-07-25 10:18:29--  https://uc89d1cc966b106f9fc433469516.dl.dropboxusercontent.com/cd/0/inline/CAiBAX-iU6ncQ_H_Gp7meyAfm1gT33Msp7muonyRBtr6Xv2UzJa724skw1utaIKinFl1A

In [3]:

!!tar -xvf genres.tar.gz?dl=0 # To unzip the audio zip file

['genres/',
 'genres/blues/',
 'genres/blues/blues.00000.au',
 'genres/blues/blues.00001.au',
 'genres/blues/blues.00002.au',
 'genres/blues/blues.00003.au',
 'genres/blues/blues.00004.au',
 'genres/blues/blues.00005.au',
 'genres/blues/blues.00006.au',
 'genres/blues/blues.00007.au',
 'genres/blues/blues.00008.au',
 'genres/blues/blues.00009.au',
 'genres/blues/blues.00010.au',
 'genres/blues/blues.00011.au',
 'genres/blues/blues.00012.au',
 'genres/blues/blues.00013.au',
 'genres/blues/blues.00014.au',
 'genres/blues/blues.00015.au',
 'genres/blues/blues.00016.au',
 'genres/blues/blues.00017.au',
 'genres/blues/blues.00018.au',
 'genres/blues/blues.00019.au',
 'genres/blues/blues.00020.au',
 'genres/blues/blues.00021.au',
 'genres/blues/blues.00022.au',
 'genres/blues/blues.00023.au',
 'genres/blues/blues.00024.au',
 'genres/blues/blues.00025.au',
 'genres/blues/blues.00026.au',
 'genres/blues/blues.00027.au',
 'genres/blues/blues.00028.au',
 'genres/blues/blues.00029.au',
 'genres/b

In [4]:
!rm genres.tar.gz?dl=0

In [5]:
import os
import pickle
from glob import iglob
import numpy as np
import librosa

DATA_AUDIO_DIR = './genres'
TARGET_SR = 8000
OUTPUT_DIR = './output'
OUTPUT_DIR_TRAIN = os.path.join(OUTPUT_DIR, 'train')
OUTPUT_DIR_TEST = os.path.join(OUTPUT_DIR, 'test')
AUDIO_LENGTH = 10000

In [6]:
os.makedirs('/content/output/train/')
os.makedirs('/content/output/test/')

In [7]:
class_ids = {    # assigning numerical id values to different genres of music
    'blues': 0,
    'classical': 1,
    'country': 2,
    'disco': 3,
    'hiphop': 4,
    'jazz':5,
    'metal':6,
    'pop':7,
    'reggae':8,
    'rock':9
}

audio_Class = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']

def extract_class_id(wav_filename):
    if 'blues' in wav_filename:
        return class_ids.get('blues')
    elif 'classical' in wav_filename:
        return class_ids.get('classical')
    elif 'country' in wav_filename:
        return class_ids.get('country')
    elif 'disco' in wav_filename:
        return class_ids.get('disco')
    elif 'hiphop' in wav_filename:
        return class_ids.get('hiphop')
    elif 'jazz' in wav_filename:
        return class_ids.get('jazz')
    elif 'metal' in wav_filename:
        return class_ids.get('metal')
    elif 'pop' in wav_filename:
        return class_ids.get('pop')
    elif 'reggae' in wav_filename:
        return class_ids.get('reggae')
    else:
        return class_ids.get('rock')

In [8]:
extract_class_id('/content/genres/hiphop/hiphop.00014.au/')

4

In [9]:
def read_audio_from_filename(filename, target_sr):
    audio, _ = librosa.load(filename, sr=target_sr, mono=True)
    audio = audio.reshape(-1, 1)
    return audio

In [10]:
def convert_data():
  for audioItem in audio_Class:
    path = DATA_AUDIO_DIR + '/' + audioItem
    #print(path)
    for i, wav_filename in enumerate(iglob(os.path.join(path, '**/**.au'), recursive=True)):
        class_id = extract_class_id(wav_filename)
        audio_buf = read_audio_from_filename(wav_filename, target_sr=TARGET_SR)
        # normalize mean 0, variance 1
        audio_buf = (audio_buf - np.mean(audio_buf)) / np.std(audio_buf)
        original_length = len(audio_buf)
        print(i, wav_filename, original_length, np.round(np.mean(audio_buf), 4), np.std(audio_buf))
        if original_length < AUDIO_LENGTH:
            audio_buf = np.concatenate((audio_buf, np.zeros(shape=(AUDIO_LENGTH - original_length, 1))))
            #print('PAD New length =', len(audio_buf))
        elif original_length > AUDIO_LENGTH:
            audio_buf = audio_buf[0:AUDIO_LENGTH]
            #print('CUT New length =', len(audio_buf))

        output_folder = OUTPUT_DIR_TRAIN
        if i // 10 == 0:
            output_folder = OUTPUT_DIR_TEST

        output_filename = os.path.join(output_folder, str(i) + audioItem + '.pkl')
        print(output_filename)
        out = {'class_id': class_id,
               'audio': audio_buf,
               'sr': TARGET_SR}
        w=open(output_filename,'wb')
        pickle.dump(out,w)
        w.close()



In [11]:
convert_data()

0 ./genres/blues/blues.00012.au 240107 0.0 1.0
./output/test/0blues.pkl
1 ./genres/blues/blues.00010.au 240107 -0.0 1.0
./output/test/1blues.pkl
2 ./genres/blues/blues.00065.au 240107 0.0 1.0000001
./output/test/2blues.pkl
3 ./genres/blues/blues.00007.au 240107 0.0 1.0
./output/test/3blues.pkl
4 ./genres/blues/blues.00047.au 240107 0.0 0.99999994
./output/test/4blues.pkl
5 ./genres/blues/blues.00058.au 240107 -0.0 1.0
./output/test/5blues.pkl
6 ./genres/blues/blues.00016.au 240107 -0.0 0.9999999
./output/test/6blues.pkl
7 ./genres/blues/blues.00045.au 240107 0.0 1.0
./output/test/7blues.pkl
8 ./genres/blues/blues.00034.au 240107 0.0 1.0
./output/test/8blues.pkl
9 ./genres/blues/blues.00060.au 240107 -0.0 1.0000001
./output/test/9blues.pkl
10 ./genres/blues/blues.00023.au 240107 0.0 1.0
./output/train/10blues.pkl
11 ./genres/blues/blues.00015.au 240107 0.0 1.0
./output/train/11blues.pkl
12 ./genres/blues/blues.00000.au 240107 0.0 1.0
./output/train/12blues.pkl
13 ./genres/blues/blues.00

In [12]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Lambda,Conv1D, MaxPooling1D,Activation, Dense,BatchNormalization

from tensorflow.keras.models import Sequential
import numpy as np
import pickle
import os
from glob import glob

In [13]:
AUDIO_LENGTH = 10000  # 10000
OUTPUT_DIR = './output'
OUTPUT_DIR_TRAIN = os.path.join(OUTPUT_DIR, 'train')
OUTPUT_DIR_TEST = os.path.join(OUTPUT_DIR, 'test')

In [14]:
def m5(num_classes=5): # final model creation
    print('Using Model M5')
    m = Sequential()
    m.add(Conv1D(128,
                 input_shape=[AUDIO_LENGTH, 1],
                 kernel_size=80,
                 strides=4,
                 padding='same',
                 kernel_initializer='glorot_uniform',
                 kernel_regularizer=regularizers.l2(l=0.0001)))
    m.add(BatchNormalization())
    m.add(Activation('relu'))
    m.add(MaxPooling1D(pool_size=4, strides=None))
    m.add(Conv1D(128,
                 kernel_size=3,
                 strides=1,
                 padding='same',
                 kernel_initializer='glorot_uniform',
                 kernel_regularizer=regularizers.l2(l=0.0001)))
    m.add(BatchNormalization())
    m.add(Activation('relu'))
    m.add(MaxPooling1D(pool_size=4, strides=None))
    m.add(Conv1D(256,
                 kernel_size=3,
                 strides=1,
                 padding='same',
                 kernel_initializer='glorot_uniform',
                 kernel_regularizer=regularizers.l2(l=0.0001)))
    m.add(BatchNormalization())
    m.add(Activation('relu'))
    m.add(MaxPooling1D(pool_size=4, strides=None))
    m.add(Conv1D(512,
                 kernel_size=3,
                 strides=1,
                 padding='same',
                 kernel_initializer='glorot_uniform',
                 kernel_regularizer=regularizers.l2(l=0.0001)))
    m.add(BatchNormalization())
    m.add(Activation('relu'))
    m.add(MaxPooling1D(pool_size=4, strides=None))
    m.add(Lambda(lambda x: K.mean(x, axis=1)))  # Same as GAP for 1D Conv Layer
    m.add(Dense(num_classes, activation='softmax'))
    return m

In [39]:
def get_data(file_list):
    def load_into(_filename, x1, y1):
        #print(_filename)
        with open(_filename, 'rb') as f:
            audio_element = pickle.load(f)
            x1.append(audio_element['audio'])
            y1.append(int(audio_element['class_id']))

    x, y = [], []
    for filename in file_list:
        load_into(filename, x, y)
    return np.array(x), np.array(y)

In [43]:
num_classes = len(audio_Class)
model = m5(num_classes=num_classes)

if model is None:
    exit('Something went wrong!!')

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
print(model.summary())

Using Model M5
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_8 (Conv1D)           (None, 2500, 128)         10368     
                                                                 
 batch_normalization_8 (Batc  (None, 2500, 128)        512       
 hNormalization)                                                 
                                                                 
 activation_8 (Activation)   (None, 2500, 128)         0         
                                                                 
 max_pooling1d_8 (MaxPooling  (None, 625, 128)         0         
 1D)                                                             
                                                                 
 conv1d_9 (Conv1D)           (None, 625, 128)          49280     
                                                                 
 batch_normalization_9 (Batc  (None, 62

In [45]:
train_files = glob(os.path.join(OUTPUT_DIR_TRAIN, '**.pkl')) #training files


In [46]:
x_train, y_train = get_data(train_files)#splitting into train_X and train_Y

In [47]:
x_train, x_train.shape #checking dimension of train_X

(array([[[-0.32669958],
         [-0.39169145],
         [-0.03271423],
         ...,
         [-0.31736758],
         [-0.4384248 ],
         [-0.7013475 ]],
 
        [[-0.51258725],
         [-0.956677  ],
         [-0.9264313 ],
         ...,
         [-0.51970035],
         [-0.11188457],
         [ 0.25325286]],
 
        [[ 0.1283182 ],
         [ 1.006446  ],
         [ 1.2855532 ],
         ...,
         [ 0.02910187],
         [ 1.0265107 ],
         [ 0.710112  ]],
 
        ...,
 
        [[ 0.03651265],
         [ 0.63770664],
         [ 1.4575095 ],
         ...,
         [ 0.42427933],
         [ 1.5735946 ],
         [ 1.1581435 ]],
 
        [[-0.46868017],
         [-0.8645601 ],
         [-0.8755968 ],
         ...,
         [ 0.46480823],
         [ 0.42502147],
         [ 0.27574837]],
 
        [[-0.18396142],
         [-0.35597986],
         [-0.2795313 ],
         ...,
         [ 0.5481262 ],
         [ 0.9692883 ],
         [ 0.5195944 ]]], dtype=float32),
 (90

In [48]:
y_train, y_train.shape #checking dimension of train_Y

(array([9, 0, 0, 0, 9, 3, 4, 8, 5, 0, 6, 1, 3, 8, 8, 6, 4, 5, 1, 8, 0, 3,
        0, 7, 4, 2, 2, 0, 3, 4, 8, 2, 1, 1, 7, 9, 1, 6, 7, 0, 5, 6, 3, 7,
        7, 0, 5, 7, 6, 4, 8, 3, 4, 0, 2, 1, 8, 3, 5, 1, 4, 7, 9, 5, 7, 7,
        0, 8, 7, 3, 0, 2, 2, 0, 8, 3, 9, 1, 6, 3, 4, 3, 7, 8, 8, 4, 9, 9,
        9, 6, 4, 8, 5, 2, 9, 9, 2, 1, 9, 9, 7, 3, 0, 7, 9, 7, 5, 9, 0, 9,
        0, 7, 3, 9, 2, 1, 0, 3, 2, 4, 5, 7, 2, 0, 1, 7, 8, 3, 9, 9, 4, 1,
        5, 1, 2, 5, 2, 4, 9, 0, 4, 9, 1, 6, 6, 0, 9, 0, 1, 8, 9, 1, 2, 9,
        7, 1, 8, 1, 3, 9, 8, 8, 9, 1, 3, 8, 9, 4, 9, 5, 2, 6, 4, 7, 1, 5,
        0, 5, 7, 3, 0, 5, 7, 6, 9, 6, 8, 4, 2, 8, 6, 6, 1, 0, 3, 8, 2, 3,
        3, 6, 8, 4, 4, 3, 4, 1, 5, 8, 3, 4, 6, 2, 8, 7, 9, 3, 5, 6, 2, 8,
        7, 9, 6, 0, 2, 7, 6, 6, 0, 2, 4, 8, 2, 4, 2, 2, 5, 7, 2, 7, 3, 5,
        7, 6, 5, 8, 5, 7, 0, 1, 8, 4, 9, 5, 1, 9, 8, 6, 4, 9, 1, 4, 2, 0,
        0, 6, 2, 2, 4, 0, 6, 9, 1, 9, 6, 7, 6, 7, 3, 5, 8, 3, 3, 0, 7, 2,
        5, 4, 8, 1, 1, 7, 4, 2, 4, 4, 

In [49]:
y_train = to_categorical(y_train, num_classes=num_classes) #y_train


In [50]:
test_files = glob(os.path.join(OUTPUT_DIR_TEST, '**.pkl'))
x_test, y_test = get_data(test_files)

In [51]:
y_test = to_categorical(y_test, num_classes=num_classes)

In [62]:
# verifying the dimensions of train and test data
print('x_train.shape =', x_train.shape)
print('y_train.shape =', y_train.shape)
print('x_test.shape =', x_test.shape)
print('y_test.shape =', y_test.shape)

x_train.shape = (900, 10000, 1)
y_train.shape = (900, 10)
x_test.shape = (100, 10000, 1)
y_test.shape = (100, 10)


In [53]:
# if the accuracy does not increase over 10 epochs, reduce the learning rate by half or so.
n=0.5
reduce_lr = ReduceLROnPlateau(monitor='acc', factor= n, patience=10, min_lr=0.0001, verbose=1)
batch_size = 128
model.fit(x=x_train,
          y=y_train,
          batch_size=batch_size,
          epochs=400,
          verbose=1,
          shuffle=True,
          validation_data=(x_test, y_test),
          callbacks=[reduce_lr])

Epoch 1/400
7/8 [=========================>....] - ETA: 0s - loss: 2.1867 - accuracy: 0.2589

8/8 [==============================] - 5s 134ms/step - loss: 2.1854 - accuracy: 0.2589 - val_loss: 2.3350 - val_accuracy: 0.1600 - lr: 0.0010
Epoch 2/400
7/8 [=========================>....] - ETA: 0s - loss: 1.7159 - accuracy: 0.4029

8/8 [==============================] - 1s 80ms/step - loss: 1.7157 - accuracy: 0.4022 - val_loss: 2.2703 - val_accuracy: 0.1600 - lr: 0.0010
Epoch 3/400
7/8 [=========================>....] - ETA: 0s - loss: 1.6712 - accuracy: 0.4308

8/8 [==============================] - 1s 88ms/step - loss: 1.6723 - accuracy: 0.4300 - val_loss: 2.2929 - val_accuracy: 0.1700 - lr: 0.0010
Epoch 4/400
7/8 [=========================>....] - ETA: 0s - loss: 1.5949 - accuracy: 0.4531

8/8 [==============================] - 1s 86ms/step - loss: 1.5997 - accuracy: 0.4522 - val_loss: 2.3231 - val_accuracy: 0.1800 - lr: 0.0010
Epoch 5/400
7/8 [=========================>....] - ETA: 0s - loss: 1.4949 - accuracy: 0.4900

...

8/8 [==============================] - 1s 92ms/step - loss: 0.3142 - accuracy: 0.9433 - val_loss: 6.3988 - val_accuracy: 0.2800 - lr: 0.0010
Epoch 397/400
7/8 [=========================>....] - ETA: 0s - loss: 0.2365 - accuracy: 0.9710

8/8 [==============================] - 1s 87ms/step - loss: 0.2401 - accuracy: 0.9700 - val_loss: 4.1607 - val_accuracy: 0.3500 - lr: 0.0010
Epoch 398/400
7/8 [=========================>....] - ETA: 0s - loss: 0.1953 - accuracy: 0.9877

8/8 [==============================] - 1s 87ms/step - loss: 0.1972 - accuracy: 0.9867 - val_loss: 6.5662 - val_accuracy: 0.2800 - lr: 0.0010
Epoch 399/400
7/8 [=========================>....] - ETA: 0s - loss: 0.3607 - accuracy: 0.9408

8/8 [==============================] - 1s 90ms/step - loss: 0.3655 - accuracy: 0.9400 - val_loss: 8.8965 - val_accuracy: 0.2400 - lr: 0.0010
Epoch 400/400
7/8 [=========================>....] - ETA: 0s - loss: 0.2444 - accuracy: 0.9710

8/8 [==============================] - 1s 83ms/step - loss: 0.2453 - accuracy: 0.9700 - val_loss: 5.5771 - val_accuracy: 0.3400 - lr: 0.0010


In [54]:
predictions=model.predict(x_test)

4/4 [==============================] - 0s 8ms/step


In [55]:
predictions

array([[9.43434715e-01, 3.71921815e-05, 3.13366436e-05, 1.66001950e-08,
        9.57092794e-04, 8.92328535e-05, 4.97903638e-02, 3.87364672e-03,
        1.20495541e-04, 1.66593224e-03],
       [3.45812231e-01, 6.37801707e-01, 5.81320637e-05, 3.84994906e-13,
        9.14915077e-08, 5.47741411e-06, 1.11167348e-04, 6.49804815e-06,
        1.66093685e-07, 1.62045136e-02],
       [1.19728461e-01, 8.03721547e-01, 1.43583733e-04, 5.57639979e-08,
        6.32553053e-07, 6.96577951e-02, 8.98076905e-05, 4.53259099e-06,
        3.23820785e-07, 6.65327208e-03],
       [4.55652172e-13, 9.99998927e-01, 8.28408364e-09, 9.18556256e-19,
        2.94724572e-14, 4.15523363e-15, 1.54042894e-08, 3.50986303e-13,
        1.55434207e-11, 1.08622476e-06],
       [1.23017130e-03, 9.93000984e-01, 2.54668156e-03, 6.65384206e-11,
        3.67946456e-07, 1.70813379e-04, 6.73608447e-05, 6.94779328e-06,
        4.45535505e-07, 2.97620124e-03],
       [3.09921422e-10, 7.52634790e-08, 6.78084007e-08, 1.15255123e-11,
   

In [56]:
from sklearn.metrics import roc_auc_score

In [57]:
y_test.shape, predictions.shape

((100, 10), (100, 10))

In [58]:
roc_auc_score(y_test,predictions)

0.8057777777777779